# Generative Adversarial Network


This is a kind of reinforcement learning and unsupervised learning where two models play together. We will have a real dataset R, a model __G__ knows as *generator* which is going to create fake data that looks just like the genuine data, while **D** is the *discriminator* who is going to evaluate and calculate the differente between the real set and the set that was modified, finally, we are going to have the *I* random noise that goes into the generator as source of entropy.


<img src="https://miro.medium.com/max/1198/1*-gFsbymY9oJUQJ-A3GTfeg.png" >


R, we are going to start with the simplest possible **R**. This function takes a mean and a standard deviation and returns which provides the right shape of a sample data from a Gaussian with those parameters. 

So, given a set of data instances X and a set of labels Y, we are going to have }
+ _Generative_ models capture the joind probability P(X, Y), or just P(X) if there are no labels
+ _Discriminative_ models capture the conditional probability P(Y|X)

<img src="https://developers.google.com/machine-learning/gan/images/generative_v_discriminative.png" >

This is the general architecture of the system

<img src="https://developers.google.com/machine-learning/gan/images/gan_diagram.svg" >

Both the generator and the discrimator are neuronal networks, where the discriminator is a classifier which knows the distribution of the data and how to segment it, in the same way, it has a backpropation which uses the loss from the interactions. 

<img src="https://developers.google.com/machine-learning/gan/images/gan_diagram_discriminator.svg">

During the discriminator's training, the discriminator ignores the generator's loss and uses the discriminator's loss, in the same way, during the generator's training the generator uses its loss.

The discriminator is going to focus to classify both real data and fake from the generator, moreover, the discriminator's loss is going to penalize the discriminator for misclassifiying a real instance as fake or a fake instance as real, finally, the discriminator updates its weights using backpropagation from the discrimator's loss. 

The generator is focuses to make fake data using the feedback from the discriminator, in other words, it learns from it in order to improve its output to be the more real. 

G is integrated by the random input _I_, a neural network, the discriminator's output, generator's loss which penalizes the generator for failing to fool the discriminator.

The *loss functions* use in a GAN are:
+ __minmax loss__
+ __Wasserstein loss__

Due to we have two neural nets we can have two loss functions. 


## GAN Variations




+ *Progressive GANs*, it's generator's first layers produce very low resolution images, and subsequent layers and details. This a kind of technique faster than the others. 

+ *Conditional GANs*, in this technique we can specify the label for each generated instance, completely different from *unconditional GANs* where it produces random outputs. 

+ *Image-to-Image translation* it takes an image as input and map it to a generated output image with different properties. 

+ *CycleGAN* it lerans to transform images from one set into images that could belong to another set. 

+ *Text-to-Image Synthesis* takes text as input and produce images that plausible and described by the text. 

+ *Super-resolution* increases the resolution of images, adding detail where necessary to fill in blurry areas. 

+ *Face inpainting* the idea of this approach is to fill chunks of an image that are blacked out.

+ *Text-to-Speech* some researches explored the application of GANs to produce synthetized speech from text input. 

## Examples

### TF-GAN tutorial from Google Developers

In [4]:
# tensorflow-gan is a library for training and evaluating GANs
import tensorflow as tf
import tensorflow_gan as tfgan
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import numpy as pd 
%matplotlib inline
tf.logging.set_verbosity(tf.logging.ERROR)

ImportError: This version of TF-GAN requires TensorFlow version >= 1.14.0; Detected an installation of version 1.13.1. Please upgrade TensorFlow to proceed.

In [5]:
!pip upgrade tensorflow


ERROR: unknown command "upgrade"



In [1]:
# mean of 4.0 and a standard deviation of 1.25
def get_distribution_sampler(mu, sigma):
    return lambda n: torch.Tensor(np.random.normal(mu, sigma, (1, n)))

G, the generator is a standard feedforward graph - two hidden layers, three linear maps. The idea is G is going to receive the *uniformly* distributed data samples from *I* and somehow replicate the *normally* distributed samples from R.

In [2]:
def get_generator_input_sampler():
    return lambda m, n: torch.rand(m, n)

In [2]:
from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

import matplotlib.pyplot as plt

import sys

import numpy as np

class GAN():
    def __init__(self):
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generates imgs
        z = Input(shape=(self.latent_dim,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        validity = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, validity)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)


    def build_generator(self):

        model = Sequential()

        model.add(Dense(256, input_dim=self.latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.img_shape), activation='tanh'))
        model.add(Reshape(self.img_shape))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Flatten(input_shape=self.img_shape))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, epochs, batch_size=128, sample_interval=50):

        # Load the dataset
        (X_train, _), (_, _) = mnist.load_data()

        # Rescale -1 to 1
        X_train = X_train / 127.5 - 1.
        X_train = np.expand_dims(X_train, axis=3)

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random batch of images
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]

            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

            # Generate a batch of new images
            gen_imgs = self.generator.predict(noise)

            # Train the discriminator
            d_loss_real = self.discriminator.train_on_batch(imgs, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator
            # ---------------------

            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

            # Train the generator (to have the discriminator label samples as valid)
            g_loss = self.combined.train_on_batch(noise, valid)

            # Plot the progress
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            # If at save interval => save generated image samples
            if epoch % sample_interval == 0:
                self.sample_images(epoch)

    def sample_images(self, epoch):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("images/%d.png" % epoch)
        plt.close()


if __name__ == '__main__':
    gan = GAN()
    gan.train(epochs=30000, batch_size=32, sample_interval=200)

Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               401920    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 257       
Total params: 533,505
Trainable params: 533,505
Non-trainable params: 0
_________________________________________________________________
____

D:\Usuarios\rhaps\Anaconda3\lib\site-packages\keras\engine\training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 0.836386, acc.: 51.56%] [G loss: 0.870476]


FileNotFoundError: [Errno 2] No such file or directory: 'images/0.png'

## References

+ [1] https://skymind.ai/wiki/generative-adversarial-network-gan
+ [2] https://forum.faceswap.dev/viewtopic.php?t=146
+ [3] https://developers.google.com/machine-learning/gan